#  Performance Analysis for TensorFlow Profiling Result
This notebook guides auidences how to generate trace.json file by using TensorFlow Profiler, and then analyze it accordingly.

## Enable TensorFlow Profiler
***
This section shows users how to enable TensorFlow Profiler for both Training and Infernce

### Prerequisites

Check the TensorFlow version used in the notebook

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%run -i "../../version_check.py"

After TensorFlow 2.11, TensorFlow only generate a xplane.pb file and doesn't have trace.json file as output. 
This notebook could only support TensorFlow version 2.10 and prior.

In [ ]:
import tensorflow as tf
major_version = int(tf.__version__.split(".")[0])
minor_version = int(tf.__version__.split(".")[1])
if major_version >= 2 and minor_version > 10:
    print("This notebook only support TensorFlow version 2.10 and pior")

### Enable TensorFlow Profiler in the word embeddings workload from Keras-IO

Get the pretrained word embeddedings workload from Keras IO

In [ ]:
!wget https://raw.githubusercontent.com/keras-team/keras-io/master/examples/nlp/pretrained_word_embeddings.py

Enable TensorFlow Profiler for both Training and Inference sessions in the word embeddings workloads.  
You could see the changes in profile.patch for both training session around line 280 and infernece session around line 297.  
We use tf.keras.callbacks.TensorBoard for training session and tf.profiler.experimental.ProfilerOptions for infernece session.  

In [ ]:
import IPython
IPython.display.Code(filename="patch/profile.patch")

Patch the pretrained_word_embeddings and enable TF Profiler.

In [ ]:
!patch < patch/profile.patch

### Run the word embeddings workload with TensorFlow Profiler

Download the glove dataset for the word embeddings workload

In [ ]:
!mkdir -p ~/.keras/datasets/; wget http://nlp.stanford.edu/data/glove.6B.zip; unzip -q glove.6B.zip; mv glove.6B.* ~/.keras/datasets/

Run the word embeddings for both training and inference.  
You will have logdir-train folder with profiling result for training, and logdir-inf folder with profiling result for inference. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%run -i "pretrained_word_embeddings.py"

Please change below Profile_Dir path for the latest inference result folder in your environment.

In [ ]:
%env Profile_Dir=logdir-inf/plugins/profile/2023_03_18_14_43_56

In [ ]:
import os
pdir = os.environ.get("Profile_Dir")
os.listdir(pdir)

Please also change the below json_path for the trace.json.gz file in the latest inference result folder.  
We will analyze the trace.json file in the following session.

In [ ]:
%env json_path=logdir-inf/plugins/profile/2023_03_18_14_43_56/icx02-tce-atsm.trace.json.gz

## Analyze TensorFlow Profiling Result
***
This section shows users how to analyze TensorFlow Profiling results with a analyze script and its HTML output file.

Analyze the json file from previous session by using Analyze/analyze script.

In [ ]:
!Analyze/analyze $json_path

Show all the report folders in order to get the full name of the latest analyze report folder.  
The current timestamp will be added as a postfix of the report folder name. 

In [ ]:
!ls | grep report_

Please change folder_dir and put the report folder that you want to analyze.  
Overall, you should pick the latest report folder.  

In [ ]:
folder_dir="report_2023-03-18_15-00/"

There are report.html file which shows all the analysis diagrams.  
We show all the analysis diagrams below, and your could refer to Analyze/README.md file for more details of those analysis diagrams. 

In [ ]:
from IPython.display import Image
from IPython.display import display
for images in os.listdir(folder_dir):
    # check if the image ends with png
    if (images.endswith(".png")):
        print(images)
        x = Image(filename=folder_dir + images)
        display(x)

The analyze script can also compare two json files from two different runs.   
We compare two json files. The stock_timeline was generated under stock tensorflow without oneDNN and mkl_timeline was generated under stock-tensorflow with oneDNN enabled.

In [ ]:
!Analyze/analyze samples/stock_timeline_resnet50v1_5_fp32_infer_merged_runs.json samples/mkl_timeline_resnet50v1_5_fp32_infer_merged_runs.json

In [ ]:
!ls | grep report_

Please change folder_dir and put the report folder that you want to analyze.  
Overall, you should pick the latest report folder.  

In [ ]:
folder_dir="report_2023-03-18_15-00/"

There are report.html file which shows all the analysis diagrams.  
We show all the analysis diagrams below, and your could refer to Analyze/README.md file for more details of those analysis diagrams. 

In [ ]:
from IPython.display import Image
from IPython.display import display
folder_dir="report_2023-03-18_15-05/"
for images in os.listdir(folder_dir):
    # check if the image ends with png
    if (images.endswith(".png")):
        print(images)
        x = Image(filename=folder_dir + images)
        display(x)


The output(both stdout and stderr) is displayed on the command line console

In [ ]:
print('[CODE_SAMPLE_COMPLETED_SUCCESFULLY]')